### Importing libraries

In [5]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

### Verifying raw data path
If assertion fails, download the Netflix Prize data from [kaggle](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data/data) and put the files in data/raw.

In [6]:
assert os.path.exists("data/raw/combined_data_1.txt")